# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lihue,21.9789,-159.3672,299.92,79,100,6.17,US,1731278602
1,1,bethel,41.3712,-73.4140,285.94,59,100,0.89,US,1731278603
2,2,stanley,54.8680,-1.6985,280.47,89,0,3.09,GB,1731278605
3,3,whitehorse,60.7161,-135.0538,273.16,74,100,5.66,CA,1731278606
4,4,adamstown,-25.0660,-130.1015,295.21,71,100,5.51,PN,1731278608


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df
filtered_cities_df = filtered_cities_df.loc[(filtered_cities_df["Max Temp"] < 350) & (filtered_cities_df["Max Temp"] > 300)]
filtered_cities_df = filtered_cities_df.loc[(filtered_cities_df["Cloudiness"] == 0)]
filtered_cities_df = filtered_cities_df.loc[(filtered_cities_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,mermoz boabab,14.7065,-17.4758,300.03,69,0,4.12,SN,1731278685
114,114,bubaque,11.2833,-15.8333,302.32,62,0,3.36,GW,1731278736
231,231,dwarka,22.2394,68.9678,300.29,66,0,4.49,IN,1731278881
283,283,marlin,31.3063,-96.8980,300.07,49,0,1.40,US,1731278943
404,404,quiindy,-25.9667,-57.2667,300.46,38,0,1.40,PY,1731279082
463,463,tartagal,-22.5164,-63.8013,300.63,46,0,2.07,AR,1731279152
517,517,ingeniero guillermo n. juarez,-23.9000,-61.8500,305.64,24,0,3.61,AR,1731279215
550,550,ojinaga,29.5667,-104.4167,302.36,8,0,0.00,MX,1731279255


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(filtered_cities_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
69,mermoz boabab,SN,14.7065,-17.4758,69,
114,bubaque,GW,11.2833,-15.8333,62,
231,dwarka,IN,22.2394,68.9678,66,
283,marlin,US,31.3063,-96.8980,49,
404,quiindy,PY,-25.9667,-57.2667,38,
463,tartagal,AR,-22.5164,-63.8013,46,
517,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,24,
550,ojinaga,MX,29.5667,-104.4167,8,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories" : categories,
    "apiKey" : geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle: {lat}, {lng}, {radius}'
    params["bias"] = f'proximity: {lat}, {lng}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mermoz boabab - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
marlin - nearest hotel: No hotel found
quiindy - nearest hotel: No hotel found
tartagal - nearest hotel: No hotel found
ingeniero guillermo n. juarez - nearest hotel: No hotel found
ojinaga - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
69,mermoz boabab,SN,14.7065,-17.4758,69,No hotel found
114,bubaque,GW,11.2833,-15.8333,62,No hotel found
231,dwarka,IN,22.2394,68.9678,66,No hotel found
283,marlin,US,31.3063,-96.8980,49,No hotel found
404,quiindy,PY,-25.9667,-57.2667,38,No hotel found
463,tartagal,AR,-22.5164,-63.8013,46,No hotel found
517,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,24,No hotel found
550,ojinaga,MX,29.5667,-104.4167,8,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)